# Final Project  N-Body Gravitational Problem.  

To simulate the N-body problem I had to solve the following lagrangian:
$$
\mathcal{L} = T - V
$$

$$
T = \sum_{i=1}^N \frac{1}{2} m_i \dot{\mathbf{r}}_i^2
$$

$$
V = -G \sum_{i=1}^{N} \sum_{j=i+1}^{N} \frac{m_i m_j}{|\mathbf{r}_i - \mathbf{r}_j|}
$$

$$
\mathcal{L} = \sum_{i=1}^N \frac{1}{2} m_i \dot{\mathbf{r}}_i^2 + G \sum_{i=1}^{N} \sum_{j=i+1}^{N} \frac{m_i m_j}{|\mathbf{r}_i - \mathbf{r}_j|}
$$

Now, to get to this lagrangian their were a couple of core assumptions I made.  No interstella drag force, no gravitational effects of relativity.  I also didn't take into account the Roche radius and instead just opted to treat each mass as a point particle and have them bounce off of each other when making contact.  I would like to implement said roche radius but I'm sure that would balloon the computational power required and the amount of time this would take.  Also just sticking to a 2d plane, 3d wouldn't be that much harder to implement but I know what the 2d results of some systems look like so I decided to use that limitation. So I just stuck bodies with masses and the gravitational force on them.  With all that assumed I solved the lagrangian to find.  

$$
\frac{d}{dt} \left( \frac{\partial \mathcal{L}}{\partial \dot{\mathbf{r}}_i} \right) - \frac{\partial \mathcal{L}}{\partial \mathbf{r}_i} = 0
$$

$$
m_i \ddot{\mathbf{r}}_i = - \frac{\partial V}{\partial \mathbf{r}_i}
$$

$$
\Rightarrow m_i \ddot{\mathbf{r}}_i = G \sum_{\substack{j = 1 \\ j \ne i}}^{N} m_i m_j \frac{\mathbf{r}_j - \mathbf{r}_i}{|\mathbf{r}_j - \mathbf{r}_i|^3}
$$

$$
\boxed{
\ddot{\mathbf{r}}_i = G \sum_{\substack{j = 1 \\ j \ne i}}^{N} m_j \frac{\mathbf{r}_j - \mathbf{r}_i}{|\mathbf{r}_j - \mathbf{r}_i|^3}
}
$$

Now converting all those r's back to x and y as avoiding polar coordinates in the n-body problem seems easier.  
$$
\mathbf{r}_j - \mathbf{r}_i = 
\begin{bmatrix}
x_j - x_i \\
y_j - y_i
\end{bmatrix}
$$
$$
|\mathbf{r}_j - \mathbf{r}_i| = \sqrt{(x_j - x_i)^2 + (y_j - y_i)^2}
$$

Doing that will leave use with a set of equations for x and y that we can be used to find a numeric solution to the N-Body gravitational problem, given our constraints.  

I've also include the option to use four different integrators.  Three of which are selectable integrators from scipy.ivp.  Those are RK45, DOP853, and Radau.  I also included ode.  Now, due to the time it takes to generate each simulation, and its animation, as well as the amount of time taken to trouble shoot, I can't be certain that everything works without issue.  For example, I know that when it is told to do the random simulation it has failed to produce things on a number of occasions, but I think it's working now.  Aso, due to the interactive nature of this, it's not really set up to just run it all and talk about the results.

I tried to make it so I didn't have to assume the size.  When creating the premade simulations I just stuck to the larger bodies, I didn't include moons, asteroids, etc.  I believe the moddel could handle them, but I'm unsure if my computer could handle the increase in complexity without making this program utilize multiple cores or cuda implemintation. 


In [ ]:
#All library imports go here.  
import numpy as np
from scipy.integrate import solve_ivp
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from NBodySimulatorClass import NBodySimulator
from NBodySimulatorClass import Get_User_input
from AnimationClass import NBodyAnimation
from IPython.display import HTML
#ran into size limits, this should fix that. I have changed how many points are simulated so I may not need this change anymore, but going to leave it  
import matplotlib as mpl
mpl.rcParams['animation.embed_limit'] = 200  # Set limit to 200 MB

## Lets talk about some of the import choices. 
  I chose to create and import several classes.  The Nbodysimulatorclass handles all the integration and ploting.  I also stored my user input function in there as well.  Probably could have made that a seperate class, or kept it in this notebook, but it seemed like a good idea at the time and I'm afraid I'll break something if I change it.  The plotting is alos included in that class.  I really should have included it in the animation class, but I originally tried to get manon to work so it made sense to keep them seperate.  It's not going to get changed because it works, and I don't want to break anything.  Probably could have combined the two classes, but the longer a document gets the harder I find it to manage so seperating them so I could have them open in different windows worked better for me.  Also I had an issue with running into memory limits, so I increased the amount of memory an animation was allowed to use.  I have still seen simulation choice 2 get up to 150ish megs so it's best to keep that in.  

## Now about that function below. 

The first function below converts our user input and sets our time scale.  I have set this simulation to accept inputs in days, as I find that more intuitive with the scale I was simulating, but I left G in seconds.  So I needed a place to convert days to seconds, and this also seemed like a good place to set the time scale for integration.  
I was having problems with it taking forever to simulate, it was to the point where I was trying to impliment multi-core calculations, and was thinking about cuda implimentation.  I chose against it, partially because it was eating into my time, and partially because I didn't know what the end user would be using so it felt like it was taking more time than it would save.  
The simulations where to long though, then I realized I was using np.linspace instead of np.arange.  As we can see in the example below running a simulation that is 100,000 days, has 10,000 points for np.arange, but 864000 for np linespace.  Currently it should be set up to cap out at 10000 points no matter how many days the simulation is, but I'm fairly certain the higher the day number the less precise the simulatino will become due to this.  Though before that change I managed to have this thing eat through my 32gb of ram and saturate my m.2 so its a good change.  

In [ ]:
# Sample code
d =100000
t_span = 86400*d  # seconds
t_step = int(np.ceil((d/100)*864)) # This should make it so that at most I'll get only 10000 points to evalulate.  
t_eval = np.arange(0, t_span, t_step)
t_example = np.linspace(0,t_span,t_step)
x = len (t_eval)
y = len (t_example)
print(x,y)

In [ ]:
#Converter and time span setter. 
def time_function(d):
    '''
    Okay, should take days and spit back my t_span, t_step, and t_eval
    '''
    t_span = 86400*d  # seconds
    t_step = int(np.ceil((d/100)*864)) # This should make it so that at most I'll get only 10000 points to evalulate.  Add a 0 to 864 to reduce no. of points and speed simulation.  
    t_eval = np.arange(0, t_span, t_step)
    return t_span, t_step, t_eval

##### After some trial and error I decided the best way to handle the getting of the user inputs and generating the output was to put all that into a function.  In my mind this looks a bit clunky but it allows it to be used multiple times with little issue.  

Also, while ode integrator is included the step size compared to the duration is often insufficient.  I believe this is what causes the jaggy nature of the plots an animations when I try to use that integrator.  I could like alter it to increase the number of points, but simulations are already taking a little bit, and I find that the current max of 10,000 points seems to be sufficient for the other integrators to work.    

In [ ]:
#Gets user inputs.  
def User_Inputs():
    '''
    Requires no initial input when called.  
    Will prompt user for their own inputs, or allow them to select from several premade simulations.
    The random simulation only works sometimes.  
    '''
    Si = 0
    masses = 0
    positions = np.array([0,0])
    velocities = np.array([0,0])
    days = 0 
    G = 6.67e-11
    Y = str(input('would you like to input your own data? Y/N'))
    if Y == 'Y' or Y == 'y':
        masses, positions, velocities, days, integrator, G = Get_User_input()
    else:
        Si = int(input('Which simulation would you like to see?\n'
            '1: A simulation of our galaxy?\n'
            '2: Our galaxy if a new star appeared at the edge of it?\n'
            '3: Inner solar system only? \n'
            '4: Something random? (due to random nature it may fail.  Use at own risk)\n'))
        integrator = str(input('please select your integrator: R for RK45, D for DOP853, A for Radau, O for ode'))
    if Si == 0:
        sim = NBodySimulator(masses, positions, velocities, Gravity = G)
        t_span, t_step, t_eval = time_function(days)

    if Si == 1:
        # All the planets in the solar system.  No moons, or other debri.  
        days = 10000
        masses = [1.99e30, 3.30e23, 4.87e24, 5.97e24, 6.42e23, 1.90e27, 5.68e26, 8.68e25, 1.02e26]
        positions = np.array([[0,0], [5.79e10,0], [1.08e11,0], [0,1.50e11], [0,2.28e11], [7.78e11,0], [1.43e12,0], [2.87e12,0], [4.50e12,0]])
        velocities = np.array([[0,0], [0,4.79e4], [0,3.5e4], [2.98e4,0], [2.41e4,0], [0, 1.31e4], [0,9.7e3], [0,6.8e3], [0,5.43e3]])
        t_span, t_step, t_eval = time_function(days)
        sim = NBodySimulator(masses, positions, velocities, Gravity = G)
        
    if Si == 2:
        #I like this one.
        days = 6000
        masses = [1.99e30, 3.30e23, 4.87e24, 5.97e24, 6.42e23, 1.90e27, 5.68e26, 8.68e25, 1.02e26,1.99e32]
        positions = np.array([[0,0], [5.79e10,0], [1.08e11,0], [0,1.50e11], [0,2.28e11], [7.78e11,0], [1.43e12,0], [2.87e12,0], [4.50e12,0],[5e12,8e5]])
        velocities = np.array([[0,0], [0,4.79e4], [0,3.5e4], [2.98e4,0], [2.41e4,0], [0, 1.31e4], [0,9.7e3], [0,6.8e3], [0,5.43e3], [-2e3,0]])
        t_span, t_step, t_eval = time_function(days)
        sim = NBodySimulator(masses, positions, velocities, Gravity = G)
    
    if Si == 3:
        #Only doing 4 planets, and the sun, gives us a more zoomed in picture.  
        days = 1000
        masses = [1.99e30, 3.30e23, 4.87e24, 5.97e24, 6.42e23]
        positions = np.array([[0,0], [5.79e10,0], [1.08e11,0], [0,1.50e11], [0,2.28e11]])
        velocities = np.array([[0,0], [0,4.79e4], [0,3.5e4], [2.98e4,0], [2.41e4,0]])
        t_span, t_step, t_eval = time_function(days)
        sim = NBodySimulator(masses, positions, velocities, Gravity = G)
        
        
    if Si == 4:
        #This should generate a random simulation and simulate it for a much shorter time.  
        #This also fails somewhat consistently, not going to remove it and don't have the time to trouble shoot.
        #often times only two bodies do anything interesting, and that's only really visible in the state space plots.  
        days =.05
        masses = [5.0, 5.0, 5.0, 5.0]
        positions = np.random.randint(-100, 101, size=(4, 2))   
        velocities = np.random.randint(-100, 101, size=(4, 2))  
        t_span, t_step, t_eval = time_function(days)
        sim = NBodySimulator(masses, positions, velocities, Gravity=9000)

    #Uses selected integrator on our sim class.    
    if integrator == 'R' or integrator == 'r':
        sol = sim.ivp_solve((0, t_span), t_eval, 'RK45' )
    if integrator == 'D' or integrator == 'd':
        sol = sim.ivp_solve((0, t_span), t_eval, 'DOP853')
    if integrator == 'A' or integrator == 'a':
        sol = sim.ivp_solve((0, t_span), t_eval, 'Radau')
    #I'm having lots of problems with ode integrator right now.  It doesn't handle the animations well, and has some annoying artifacting.
    #also the last integrator I decided to add.  I'll leave it in since it works for most of the other bits. 
    if integrator == 'O' or integrator == 'o':
        sol = sim.ode_solve(t_eval)

    #This is here to make the animation not take forever.  Might not be needed anymore since the changes to the time function.   
    n = len(masses)
    frames = sol.y[:2 * n].reshape((n, 2, -1))     # shape: (n, 2, time)
    no_of_frames = len(frames[0, 0])
    frame_stride = 1
    while  no_of_frames > 2000:
        no_of_frames = no_of_frames / 10
        frame_stride = frame_stride*10
    #was included to make something work.  Forget exactly what.  
    if frame_stride > sim.solution.t.size:
        frame_stride = max(1, sim.solution.t.size // 10)
    
    anim = NBodyAnimation(sim,frame_stride=frame_stride)
    sim.plot("n-body simulation")
    #for some reason it won't display the plot and animation at the same time but passing the anim class out works.  
    return anim
 
    

#### I was having issues getting the above function to display the graph and animation when called, so I settled for the below as a workaround.  Given more time I would solve this issue, but each time I run the simulation it takes a couple of minutes.  I don't really want to try to fix this so close to the due time when I have a workable solution.    

In [ ]:
anim = User_Inputs()
anim.animate()

In [ ]:
#This allows one to save an animation.  I wanted to include it with the call to make the animation, but that caused issues. 
S = str(input('would you like to save your animation? Y/N'))
if S == 'y' or S == 'Y':
    SN = str(input('what would you like it be saved as? Do not include file extension'))
    anim.save_animation(f"{SN}.mp4", fps=60)

#### This below simulation I thought was neat.  I would like to note that gravity is such a week force that if our sun where to disapear suddenly it appears that our planets would just continue at their last heading.  So I had to increase gravity by 6 magnitudes to get the planets to interact in an interesting way. That is what is simulated below.  

In [ ]:
# This is 100 day animation of our four inner planets, no sun, and higher gravity.  
days = 100
masses = [3.30e23, 4.87e24, 5.97e24, 6.42e23]
positions = np.array([[5.79e10,0], [1.08e11,0], [0,1.50e11], [0,2.28e11]])
velocities = np.array([[0,4.79e4], [0,3.5e4], [2.98e4,0], [2.41e4,0]])
t_span, t_step, t_eval = time_function(days)
sim = NBodySimulator(masses, positions, velocities, Gravity = 6.67e-5)
sol = sim.ivp_solve((0, t_span), t_eval, 'RK45' )
n = len(masses)
frames = sol.y[:2 * n].reshape((n, 2, -1))     # shape: (n, 2, time)
no_of_frames = len(frames[0, 0])
frame_stride = 1
while  no_of_frames > 2000:
    no_of_frames = no_of_frames / 10
    frame_stride = frame_stride*10

anim = NBodyAnimation(sim,frame_stride=frame_stride)
sim.plot("n-body simulation")
anim.animate()

In [ ]:
S = str(input('would you like to save your animation? Y/N'))
if S == 'y' or S == 'Y':
    SN = str(input('what would you like it be saved as? Do not include file extension'))
    anim.save_animation(f"{SN}.mp4", fps=60)